In [43]:
import numpy as np
import pandas as pd
import datetime

In [44]:
data3=pd.read_csv("Popularity Index.csv")

In [45]:
data3.head()

,Destination,Condition,Month,Temperature,Festival,Holiday Season,Popularity Index
0,Bengaluru,Clear,January,18.5,Yes,Yes,61
1,Udaipur,Clear,January,24.9,Yes,Yes,61
2,Jodhpur,Clear,January,24.8,Yes,Yes,46
3,Ooty,Clouds,January,3.4,Yes,Yes,48
4,Madurai,Rain,January,21.5,Yes,Yes,59


In [46]:
x=data3.iloc[:,0:6]
y=data3[["Popularity Index"]]

In [47]:
from sklearn.preprocessing import LabelEncoder
categorical_cols = ["Destination", "Condition", "Month", "Festival", "Holiday Season"]
numeric_cols = ["Temperature"]  

# Encode only categorical
label_encoders = {}
for column in categorical_cols:
    le = LabelEncoder()
    x[column] = le.fit_transform(x[column])
    label_encoders[column] = le

# Numeric columns stay as they are
x[numeric_cols] = x[numeric_cols]

In [48]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=0)

In [49]:
from sklearn.preprocessing import StandardScaler
st=StandardScaler()
x_train=st.fit_transform(x_train)
x_test=st.transform(x_test)

In [50]:
import xgboost as xgb
xgb_reg = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=10,
    random_state=42
)

In [51]:
xgb_reg.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [52]:
import requests

api_key = "1f83e6304f292db0a23225da51a6f712"  
city_name = "Agra"

base_url = "https://api.openweathermap.org/data/2.5/weather?"
complete_url = f"{base_url}q={city_name}&appid={api_key}&units=metric"

response = requests.get(complete_url)
if response.status_code == 200:
    data_api = response.json()
    condition = data_api['weather'][0]['main']  # e.g., "Clear", "Rain"
    temperature = data_api['main']['temp']      # already in Celsius
    wind_speed = data_api['wind']['speed'] * 3.6  # convert m/s → km/h
else:
    raise Exception("API request failed")

In [53]:
print(condition)
print(temperature)
print(wind_speed)

Clear
34.17
10.044


In [54]:
input={
    "Destination":city_name,
    "Condition":condition,
    "Month":datetime.date.today().strftime("%B"),
    "Temperature":temperature,
    "Festival":"No",
    "Holiday Season":"Yes"
}

In [55]:
input_df=pd.DataFrame([input])
# Encode categorical only
for column in categorical_cols:
    encoder = label_encoders[column]
    input_df[column] = encoder.transform(input_df[column])

# Keep numeric as-is
input_df[numeric_cols] = input_df[numeric_cols]
input=st.transform(input_df)

In [56]:
prediction=xgb_reg.predict(input)
print(int(prediction[0]))

38


In [57]:
y_pred=xgb_reg.predict(x_test)

In [58]:
y_pred[0:5]

array([46.197987, 56.72219 , 55.608135, 48.616383, 54.222416],
      dtype=float32)

In [59]:
y_test.head()

,Popularity Index
398,47
3833,57
4836,54
4572,52
636,53


In [60]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
mean_absolute_error(y_pred,y_test)

2.230971336364746

In [61]:
r2_score(y_pred,y_test)

0.924510657787323

In [62]:
data3["Popularity Index"].describe()

count    5000.000000
mean       50.000800
std        13.151813
min        12.000000
25%        41.000000
50%        50.000000
75%        60.000000
max        82.000000
Name: Popularity Index, dtype: float64

In [63]:
import joblib
joblib.dump(xgb_reg, "popularity_model.pkl")
joblib.dump(st, "popularity_scaler.pkl")
joblib.dump(label_encoders, "popularity_encoders.pkl")
# Removed y_label dump since it is not defined and not needed

['popularity_encoders.pkl']